## Saving and Loading models
> a variety of use cases regarding the saving and loading of PyTorch models
- `torch.save` : Saves a serialized object to disk. This function uses Python’s pickle utility for serialization. Models, tensors, and dictionaries of all kinds of objects can be saved using this function.
- `torch.load` : Uses pickle’s unpickling facilities to deserialize pickled object files to memory. This function also facilitates the device to load the data into (see Saving & Loading Model Across Devices).
- `torch.nn.Module.load_state_dict` : Loads a model’s parameter dictionary using a deserialized state_dict. 
- `state_dict`:  A state_dict is simply a Python dictionary object that maps each layer to its parameter tensor. 

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   

In [4]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(X)
        return x 
    

In [5]:
    # Initialize model 
    model = TheModelClass()
    
    optimizer = optim.SGD(model.parameters(), lr=.001, momentum=.9)
    
    print('Model\'s state_dict:' )
    for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())
        
    print('Optimizer \'s state_dict:')
    for var_name in optimizer.state_dict():
        print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer 's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [1693947306488, 1693947309728, 1693946498000, 1693946497784, 1693946497640, 1693946496560, 1693946497856, 1693946498720, 1693946498936, 1693946498432]}]


## Saving & Loading model for inference

#### Save 

In [14]:
torch.save(model.state_dict(), './model/model.pth')

#### Load 

In [17]:
model = TheModelClass()
model.load_state_dict(torch.load('./model/model.pth'))
model.eval()

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### Save/Load Entire Model 

#### Save 

In [19]:
torch.save(model, './model/etr_model')

#### Load

In [20]:
model = torch.load('./model/etr_model')
model.eval()

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### Saving & Loading a General checkpoint for inference and/or resuming training

#### Save

In [23]:
torch.save({
    'epoch': 10,
    'model_state_dict':model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict(),
    'loss':nn.CrossEntropyLoss()
}, './model/model_ckpt')

In [24]:
model = TheModelClass()
#optimizer = TheOptimizerClass()

checkpoint = torch.load('./model/model_ckpt')
model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()


TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### Saving Multiple Models in One File 

#### Save 

In [ ]:
torch.save({
            'modelA_state_dict': modelA.state_dict(),
            'modelB_state_dict': modelB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict()
            }, './model/mul_model')

#### Load

In [ ]:
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()

### Warmstarting Model Using parameters from a different model 

#### Save

In [ ]:
torch.save(modelA.state_dict(), PATH)

#### Load 

In [ ]:
modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)

### Saving & Loading Model across devices

In [ ]:
# save
torch.save(model.state_dict(), PATH)

In [ ]:
# load
device = torch.device('cpu')
model = TheModelClass()
model.load_state_dict(torch.load(PATH, map_location=device))

### Save on GPU, Load on GPU

In [ ]:
# save
torch.save(model.state_dict(), PATH)

In [ ]:
# load 
device = torch.device('cuda')
model = TheModelClass()
model.load_state_dict(torch.load(PATH))
model.to(device)

### Save on CPU, Load on GPu

In [ ]:
# save
torch.save(model.state_dict(), PATH)

In [ ]:
# load 
device = torch.device("cuda")
model = TheModelClass()
model.load_state_dict(torch.load(PATH, map_location='cuda:0'))
model.to(device)

### Saving `torch.nn.DataParallel` models 

In [ ]:
# save
torch.save(model.module.state_dict(), PATH)

In [ ]:
# load 
# load to whatever device i want!